In [ ]:
import pandas as pd
import numpy as np
import ast
from tqdm import tqdm
import swifter
import json

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split

tqdm.pandas()

/opt/miniconda3/envs/GDSvenv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
test_df = pd.read_csv('data/news_sample_6.csv')

In [10]:
df = pd.read_csv('data/995,000_row_cleaned.csv')

/var/folders/z1/hf1tn63x7tl1qjxy1md9xy4w0000gn/T/ipykernel_50249/1234656882.py:1: DtypeWarning: Columns (0,1) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('data/995,000_row_cleaned.csv')


In [ ]:
# Remove rows with ivalid values
label_map = {"fake": 1, "satire":1, "conspiracy": 1, "unreliable": 1, "bias": 1, "rumor": 1, "junksci": 1, "hate": 1,
             "reliable": 0,  "clickbait": 0,   "political": 0}
df = df[df["type"].isin(label_map.keys())]  # Keep only rows with valid labels
print(df["type"].unique())


['political' 'fake' 'satire' 'reliable' 'conspiracy' 'unreliable' 'bias'
 'rumor' 'clickbait' 'hate' 'junksci']
['political' 'fake' 'satire' 'reliable' 'conspiracy' 'unreliable' 'bias'
 'rumor' 'clickbait' 'hate' 'junksci']


In [15]:
# Label all to either 1 or 0
labels = df["type"].map(label_map)

# Using Bag of Words
texts = df["content"]
vectorizer = CountVectorizer(max_features=10000)  # Limit to top 10,000 words
X = vectorizer.fit_transform(texts)

In [ ]:
# Load the dataset with mapped labels
df = pd.read_csv('data/news_sample_7.csv')

# Separate the two classes
type_0 = df[df['type'] == 0]
type_1 = df[df['type'] == 1]

# Check class sizes
count_0 = len(type_0)
count_1 = len(type_1)

# Identify minority and majority classes
if count_0 < count_1:
    # Type 0 is minority
    min_count = count_0
    majority_class = type_1
    minority_class = type_0
    print("Type 0 is minority")
else:
    # Type 1 is minority
    min_count = count_1
    majority_class = type_0
    minority_class = type_1
    print("Type 1 is minority")

# Undersample majority class
majority_undersampled = majority_class.sample(n=min_count, random_state=42)

# Combine the undersampled majority with minority
balanced_df = pd.concat([majority_undersampled, minority_class])

# Shuffle the data
balanced_df = balanced_df.sample(frac=1, random_state=42).reset_index(drop=True)

# Verify the new distribution
print("\nBalanced distribution:")
print(balanced_df['type'].value_counts())

# Save the balanced dataset
balanced_df.to_csv('data/news_sample_8.csv', index=False)

In [ ]:
# Load the balanced dataset
balanced_df = pd.read_csv('data/news_sample_8.csv')

# First split: 90% temp, 10% test
temp_df, test_df = train_test_split(balanced_df, test_size=0.1, random_state=42, stratify=balanced_df['type'])

# Second split: 8/9 of temp for train (80% of original), 1/9 of temp for validation (10% of original)
train_df, val_df = train_test_split(temp_df, test_size=1/9, random_state=42, stratify=temp_df['type'])

# Verify the sizes
total = len(balanced_df)
print(f"Total samples: {total}")
print(f"Train samples: {len(train_df)} ({len(train_df)/total:.2%})")
print(f"Validation samples: {len(val_df)} ({len(val_df)/total:.2%})")
print(f"Test samples: {len(test_df)} ({len(test_df)/total:.2%})")

# Verify distribution of classes in each split
print("\nClass distribution:")
print("Train:", train_df['type'].value_counts(normalize=True))
print("Validation:", val_df['type'].value_counts(normalize=True))
print("Test:", test_df['type'].value_counts(normalize=True))

# Save splits to CSV files
train_df.to_csv('data/train_data.csv', index=False)
val_df.to_csv('data/val_data.csv', index=False)
test_df.to_csv('data/test_data.csv', index=False)

(903679, 10000)
